In [1]:
from pathlib import Path
from book_loader import BookLoader
from typing import Dict, List

In [2]:
def make_data_path(data_dir: str, data_fn: str) -> Path:
    data_path = Path(data_dir + data_fn).expanduser().resolve()
    assert data_path.exists()
    return data_path

In [3]:
data_path = make_data_path("data/", "D5627-Dolan.docx")
book = BookLoader(data_path, tables_of_contents_idx=152)
chapters: Dict[int, List[str]] = book.chapters

/home/ahmedm/anaconda3/envs/LDS/lib/python3.9/site-packages/simplify_docx/iterators/generic.py:193: UnexpectedElementWarning: Skipping unexpected tag: {http://schemas.openxmlformats.org/wordprocessingml/2006/main}pict
  warn("Skipping unexpected tag: %s" % (current.tag),


In [8]:
# TODO: Voir s'il est nécessaire/utile de gérer la discontinuation entre pages
print('\n'.join(chapters[0]))
# print(chapters[0])

Nous croyons que cette nouvelle édition paraît à point nommé, car le monde organisationnel est soumis à des exigences de productivité histo- riques, exigences qui demeurent, faut-il l'admettre, complexes à conju- guer avec une promotion et un soutien de la santé psychologique au travail. Si certains cadres envisagent stratégiquement d'accroître la pro- ductivité qualitative et quantitative de leur personnel en les poussant à la limite de leurs capacités, trop souvent ces mêmes gestionnaires ont de la difficulté à entrevoir les effets humains collatéraux de cette quête productiviste. Ils ne voient pas (ou simplement, sous-estiment) l'inci- dence directe qu'a sur leur propre vie et leur santé les pressions psycho- logiques qu'ils subissent ainsi que celles qu'ils font conséquemment subir à leur équipe de travail. Il serait évidemment utopique d'espérer aujour- d'hui rencontrer des contextes de travail exempts de toutes formes de stress. Le père du stress, Hans Selye, l'exprimait assez si

In [5]:
# Number of words per chapter

